In [54]:
import sagemaker as sage
import pandas as pd
from time import gmtime, strftime
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch
import os
import numpy as np
import shutil
import pandas as pd   
import numpy as np
import os
import shutil

# data prepare

In [55]:
def get_data_direct(excelname):
    jsonObj = pd.read_excel(excelname)
    jsonObj['review_content'] = jsonObj['review_content'].fillna(method='ffill')
    jsonObj = jsonObj[-jsonObj['review_content'].isnull()]
    jsonObj['review_content'] = jsonObj['review_content'].apply(lambda x: x.replace('\n',' '))
    #print (jsonObj.head())
     
    #readjust key
    key_dict, values, keys = get_key()
    #capitalize
    jsonObj['aspect_category'] = jsonObj['aspect_category'].map(lambda x: str(x).capitalize())
    #map
    jsonObj['aspect_category'] = jsonObj['aspect_category'].map(lambda x:convert_key(x, key_dict, values, keys))
    jsonObj = jsonObj[jsonObj['aspect_category']!='wrong']
    
    #print (jsonObj.head())
    print ("<< aspect category number: ", jsonObj.groupby('aspect_category')['aspect_category'].nunique())
    
    #generate label 
    #jsonObj['label'] = jsonObj.apply(lambda row: (str(row['aspect_term']),str(row['aspect_category']),str(row['opinion_term']),str(row['sentiment_polarity'])), axis=1)
    jsonObj['label'] = jsonObj.apply(lambda row: get_label(row), axis=1)
    
    jsonObj = jsonObj[['review_content','label']]
    
    #print (jsonObj.head())
    print ("number of rows label is null", len(jsonObj[jsonObj['label'].isnull()]))
    jsonObj = jsonObj[-jsonObj['label'].isnull()]
    
    #agg label
    jsonObj_grouped = jsonObj.groupby('review_content')['label'].apply(list).reset_index()
    
    #print (jsonObj['review_content'].nunique())
    #print (jsonObj_grouped.head)
    #print (jsonObj_grouped.shape)
    return jsonObj_grouped

In [56]:
df = get_data_direct('../../data/GPT打标（带描述）-Jackie.xlsx')

<< aspect category number:  aspect_category
Accessory prices                      1
App interactive control experience    1
App interface design                  1
Appearance design                     1
Battery capacity                      1
                                     ..
Trade-in policy                       1
User manual                           1
Waterproof                            1
Wireless charging                     1
Workmanship craftsmanship             1
Name: aspect_category, Length: 70, dtype: int64
number of rows label is null 0


In [57]:
train, validate, test = np.split(df.sample(frac=1), [int(.8*len(df)), int(.9*len(df))])
train = pd.concat([df,train])

In [58]:
type(train)

pandas.core.frame.DataFrame

In [59]:
def get_key():
    category_list = pd.read_excel('../../data/GPT预打标20231023.xlsx', sheet_name='标签释义1012',header=1)
    key = list(category_list['三级指标 (En)'])
    value = list(category_list['三级指标 GPT翻译'])
    key = [x.capitalize() for x in key]
    value = [x.capitalize() for x in value]
    keydict = dict(zip(key, value))
    return keydict, value, key

#preprocess data
def write_txt(df,path):
    '''
    write back to txt
    '''
    #output txt file
    df = df.reset_index()
    with open(path,'a')as f:
        for i in range(len(df)):
            f.write("{} #### {}".format(df.loc[i,'review_content'].strip(),df.loc[i,'label']))
            f.write('\n')
            
            
def mkdir_rm(folder):
    '''
    make directory if not exists
    '''
    if os.path.exists(folder):
        shutil.rmtree(folder) 
    os.mkdir(folder)
    print ("<< path valid!")
    
def get_label(row):
    try:
        if pd.isna(row['Remark']):
            return (str(row['aspect_term']),str(row['aspect_category']),str(row['opinion_term']),str(row['sentiment_polarity']))
        elif row['Remark']=='改':
            return (str(row['x1']),str(row['x2']),str(row['x3']),str(row['x4']))  
        elif row['Remark']=='修改':
            return (str(row['x1']),str(row['x2']),str(row['x3']),str(row['x4']))
    except:
        return (str(row['aspect_term']),str(row['aspect_category']),str(row['opinion_term']),str(row['sentiment_polarity']))

def convert_key(x, key_dict, values, keys):
    if x in values:
        return x
    elif x in keys:
        return key_dict[x]
    else:
        return 'wrong'
    
def get_data(sheet):
    jsonObj = pd.read_excel('../../data/GPT预打标20231023.xlsx', sheet_name=sheet)
    jsonObj['review_content'] = jsonObj['review_content'].fillna(method='ffill')
    jsonObj = jsonObj[-jsonObj['review_content'].isnull()]
    jsonObj['review_content'] = jsonObj['review_content'].apply(lambda x: x.replace('\n',' '))
    #print (jsonObj.head())
     
    #readjust key
    key_dict, values, keys = get_key()
    #capitalize
    jsonObj['aspect_category'] = jsonObj['aspect_category'].map(lambda x: str(x).capitalize())
    #map
    jsonObj['aspect_category'] = jsonObj['aspect_category'].map(lambda x:convert_key(x, key_dict, values, keys))
    jsonObj = jsonObj[jsonObj['aspect_category']!='wrong']
    
    #print (jsonObj.head())
    print ("<< aspect category number: ", jsonObj.groupby('aspect_category')['aspect_category'].nunique())
    
    #generate label 
    #jsonObj['label'] = jsonObj.apply(lambda row: (str(row['aspect_term']),str(row['aspect_category']),str(row['opinion_term']),str(row['sentiment_polarity'])), axis=1)
    jsonObj['label'] = jsonObj.apply(lambda row: get_label(row), axis=1)
    
    jsonObj = jsonObj[['review_content','label']]
    
    #print (jsonObj.head())
    print ("number of rows label is null", len(jsonObj[jsonObj['label'].isnull()]))
    jsonObj = jsonObj[-jsonObj['label'].isnull()]
    
    #agg label
    jsonObj_grouped = jsonObj.groupby('review_content')['label'].apply(list).reset_index()
    return jsonObj_grouped

def preprocess_data(output_path,over_sample=True):  
    #remove & remake the output folder 
    mkdir_rm(output_path)
    
    #get data
    dfv1 = get_data('US-10-13-（刘莎-展诚-Nick）')
    dfv2 = get_data('3星及以下（145）-产品核验-标注-1007')
    
    print ("v1 shape {}, v1 sample {}".format(dfv1.shape,dfv1.head()))
    print ("v2 shape {} v2 sample {}".format(dfv2.shape,dfv2.head()))
    
    result = pd.concat([dfv1,dfv2])
    
    #train/test/val split
    train, validate, test = np.split(result.sample(frac=1), [int(.8*len(result)), int(.9*len(result))])
    
    #concat 2000
    df = get_data_direct('../../data/GPT打标（带描述）-Jackie.xlsx')
    train = pd.concat([df,train])
   
    print ("training size: ",train.shape)
    print ("test size: ",test.shape)
    print ("validate size: ",validate.shape)
    
    # write train/test/dev
    write_txt(train,os.path.join(output_path,'train.txt'))
    write_txt(test,os.path.join(output_path,'test.txt'))
    write_txt(validate,os.path.join(output_path,'dev.txt'))
    print ("<<<finish data preparing!")
    
output_path = './data/tasd/huabao1030'
preprocess_data(output_path,over_sample=False)

<< path valid!
<< aspect category number:  aspect_category
Accessory prices                      1
App interactive control experience    1
App interface design                  1
Appearance design                     1
Battery capacity                      1
                                     ..
Trade-in policy                       1
User manual                           1
Waterproof                            1
Wireless charging                     1
Workmanship craftsmanship             1
Name: aspect_category, Length: 66, dtype: int64
number of rows label is null 531
<< aspect category number:  aspect_category
App interface design                      1
Appearance design                         1
Battery capacity                          1
Battery cycle life                        1
Battery replacement                       1
Charging accessories                      1
Charging quality                          1
Charging speed                            1
Compatibility/adaptabili

# train

In [60]:
sess = sage.Session()

WORK_DIRECTORY = "./data"

# S3 prefix
prefix = "huabao1030"

role = get_execution_role()

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [61]:
hyperparameters = {
    "task" : "tasd", 
    "dataset" : "huabao1030", 
    "model_name_or_path" : "t5-large", 
    "paradigm": "extraction",
    "eval_batch_size" :"16",
    "train_batch_size" :"32",
    "learning_rate" :"3e-4",
    "num_train_epochs":"10",
    "n_gpu": "0",
    "do_direct_eval":True
}

In [62]:
entry_point = 'finetune.py'
source_dir = './'
git_config = None
role = get_execution_role()
framework_version = '1.7.1'
py_version='py36'
instance_type='ml.g5.4xlarge'
#instance_type='local_gpu'
instance_count=1

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [63]:
estimator = PyTorch(
    entry_point = entry_point,
    source_dir = source_dir,
    git_config = git_config,
    role = role,
    debugger_hook_config=False,
    hyperparameters = hyperparameters,
    framework_version = framework_version, 
    py_version = py_version,
    instance_type = instance_type,
    instance_count = instance_count
)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [52]:
inputs = {'tasd': data_location+'/tasd/'}

In [64]:
response = estimator.fit(inputs,job_name="level3-test-20-t5large-1030-add2000-v4")

Using provided s3_resource


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: level3-test-20-t5large-1030-add2000-v4


2023-10-30 13:35:54 Starting - Starting the training job...
2023-10-30 13:36:10 Starting - Preparing the instances for training.........
2023-10-30 13:37:47 Downloading - Downloading input data
2023-10-30 13:37:47 Training - Downloading the training image..................
2023-10-30 13:40:28 Training - Training image download completed. Training in progress....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-10-30 13:41:11,845 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-10-30 13:41:11,870 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-10-30 13:41:11,872 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-10-30 13:41:12,420 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip install -r requirements.txt
  Created wheel for jieba: f

UnexpectedStatusException: Error for Training job level3-test-20-t5large-1030-add2000-v4: Failed. Reason: ClientError: Please use an instance type with more memory, or reduce the size of job data processed on an instance.

# deploy 

In [3]:
!ls ../../data

GPT打标（带描述）-Jackie.xlsx  GPT预打标20230926.xlsx  GPT预打标20231023.xlsx


In [9]:
import sagemaker

instance_type = 'ml.m5.4xlarge'
role = sagemaker.get_execution_role()

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [32]:
s3_model = estimator.model_data 
#s3_model = "s3://sagemaker-us-east-1-726335585155/pytorch-training-2022-06-16-07-12-35-530/output/model.tar.gz"

In [33]:
s3_model

's3://sagemaker-us-west-2-726335585155/pytorch-training-2023-10-23-10-54-36-037/output/model.tar.gz'

In [11]:
!aws s3 ls s3://sagemaker-us-east-1-726335585155/pytorch-training-2022-06-16-07-12-35-530/output/model.tar.gz

2022-06-16 07:55:55 3452559113 model.tar.gz


In [12]:
from sagemaker.pytorch.model import PyTorchModel

pytorch_model = PyTorchModel(model_data=s3_model, 
                             role=role,
                             entry_point='inference.py', 
                             source_dir='./', 
                             framework_version='1.7.1', 
                             py_version='py36'
                ) # TODO set model_server_workers=1 to avoid torchhub bug

predictor = pytorch_model.deploy(instance_type=instance_type, initial_instance_count=1)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker:Repacking model artifact (s3://sagemaker-us-west-2-726335585155/pytorch-training-2023-10-23-09-15-12-985/output/model.tar.gz), script artifact (./), and dependencies ([]) into single tar.gz file located at s3://sagemaker-us-west-2-726335585155/pytorch-inference-2023-10-23-09-34-54-059/model.tar.gz. This may take some time depending on model size...
INFO:sagemaker:Creating model with name: pytorch-inference-2023-10-23-09-36-09-194
INFO:sagemaker:Creating endpoint-config with name pytorch-inference-2023-10-23-09-36-09-841
INFO:sagemaker:Creating endpoint with name pytorch-inference-2023-10-23-09-36-09-841


--------!

In [ ]:
from boto3.session import Session
import json

body = {"inputs": "Jacery 240, Small, heavy, portable, has carrying handle, cute design and built. 2 year warranty. Sadly battery is not replacaceable so when it's done it's done. Pricey at $219, however, i'm pleased with purchace. Would recommend. Wished it had some some sort of solar panel on front, back or side that recharged it as well."}

session = Session()
runtime = session.client("runtime.sagemaker")
response = runtime.invoke_endpoint(
    EndpointName=predictor.endpoint_name,
    ContentType="application/json",
    Body=json.dumps(body),
)
result = json.loads(response["Body"].read())
print (result)


In [20]:
%%time

predictor.serializer = sagemaker.serializers.JSONSerializer()
predictor.deserializer = sagemaker.deserializers.JSONDeserializer()

body = {"inputs": "I am pretty new to pickleball and finally decided to try out some different paddles."}

predictor.predict(body,initial_args={"ContentType":"application/json"})

KeyboardInterrupt: 